**To use this notebook:** Run one line at a time waiting for each cell to return before running the next cell.

In [ ]:
%pip install -q ipylab

# Panels and Widgets

### Warning for notebooks and consoles

**Do not try to await tasks returned from any ipylab methods, doing so block forever preventing further execution.**

This happens because Ipylab employs custom messages over widget comms and widget comms is blocked during cell execution (in the default kernel and server).

see [Plugins](plugins.ipynb#Example-launching-a-small-app) or [Actions](widgets.ipynb#Notification-Actions) for an example of awaiting the tasks in a coroutine.

In [1]:
import ipywidgets as ipw

import ipylab
import asyncio

app = await ipylab.App().ready()

## Panel

A `Panel` widget is the same as a `ipywidget.Box`, but with a `Title` that is used when the panel is added to the application shell.

In [2]:
panel = ipylab.Panel(children=[ipw.Dropdown()])

To quickly add the panel to the JupyterLab *shell* main area:

In [ ]:
sc = await panel.add_to_shell(mode=ipylab.InsertMode.split_right, activate=False, as_coro=True)

In [11]:
sc in panel.connections

True

In [12]:
sc in app.shell.connections

True

`sc` is a `ShellConnection`. 
It provides an `activate` method, and in further may have other features added.

In [13]:
sc.activate()  # Will activate before returning to this notebook

<Task pending name='d20753fe-801c-49de-b124-f8861dd68159' coro=<Ipylab._wrap_awaitable() running at /home/alan/ipylab/ipylab/ipylab.py:197> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[Ipylab._task_done_callback()]>

Closing the connection will remove the panel from the shell but leave the panel open.

In [14]:
sc.close()

We can put it back in the shell.

In [15]:
sc = await panel.add_to_shell(mode=ipylab.InsertMode.split_right, activate=False)

In [16]:
# closable is on the widget in the shell rather than the panel, but we can set it using set_property.
t = sc.set_property("title.closable", False)

The title label can be updated as required.

In [17]:
panel.title.label = "This panel has a dropdown"

We can close the panel and the view will disappear.

In [18]:
panel.close()

When the panel is closed sc is also closed.

In [19]:
sc

< Not ready: ShellConnection(cid='ipylab-ShellConnection|50c7eed6-9fbd-4069-a300-757e02e89ff2') >

In the case of sliders and other widgets that fit on a single line, they can even be added to the top area directly:

In [20]:
slider = ipw.IntSlider()
app.shell.add(slider, area=ipylab.Area.top)

<Task pending name='Add to shell' coro=<Ipylab._wrap_awaitable() running at /home/alan/ipylab/ipylab/ipylab.py:199> wait_for=<Task pending name='d1ab77a2-6c7b-4bb3-a11e-78665209ac1b' coro=<Ipylab._wrap_awaitable() running at /home/alan/ipylab/ipylab/ipylab.py:199> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[Ipylab._task_done_callback(), Task.task_wakeup()]> cb=[Ipylab._task_done_callback()]>

We can also remove it from the top area when we are done.

In [21]:
slider.close()

## SplitPanel
A split panel is a subclass of Panel that provides a draggable border between widgets, whose orientatation can be either horizontal or vertical.
Let's create a `SplitPanel` with a few widgets inside.

In [22]:
split_panel = ipylab.SplitPanel()
progress = ipw.IntProgress(
    value=7,
    min=0,
    max=100,
    step=1,
    description="Loading:",
    bar_style="info",
    orientation="horizontal",
    layout={"height": "30px"},
)
slider_ctrl = ipw.IntSlider(
    min=0,
    max=100,
    step=1,
    description="Slider Control:",
)

# link the slider to the progress bar
ipw.jslink((slider_ctrl, "value"), (progress, "value"))

# add the widgets to the split panel
split_panel.children = [progress, slider_ctrl]
ipw.Box(children=[split_panel], layout={"height": "100px"})

Box(children=(SplitPanel(children=(IntProgress(value=7, bar_style='info', description='Loading:', layout=Layou…

In [23]:
split_panel.title.label = "A SplitPanel "
split_panel.title.icon_class = "jp-PythonIcon"

> As an alternative to `icon_class`, a `Panel` can also use custom [icons](./icons.ipynb).

In [24]:
split_panel.add_to_shell(area=ipylab.Area.main, mode=ipylab.InsertMode.split_bottom)

<Task pending name='Add to shell' coro=<Ipylab._wrap_awaitable() running at /home/alan/ipylab/ipylab/ipylab.py:199> wait_for=<Task pending name='35126389-efa3-4bd5-8550-7aa55e705941' coro=<Ipylab._wrap_awaitable() running at /home/alan/ipylab/ipylab/ipylab.py:199> wait_for=<Future pending cb=[Task.task_wakeup()]> cb=[Ipylab._task_done_callback(), Task.task_wakeup()]> cb=[Ipylab._task_done_callback()]>

The orientation can be updated on the fly:

In [25]:
split_panel.orientation = "horizontal"

Let's put it back to `vertical`

In [26]:
split_panel.orientation = "vertical"

Just like with boxes, we can add an existing widget (the progress bar) more than once:

In [27]:
split_panel.children += (progress,)

Or add a new widget:

In [28]:
play = ipw.Play(min=0, max=100, step=1, description="Press play")
ipw.jslink((play, "value"), (slider_ctrl, "value"))
split_panel.children += (play,)

## Left and Right Areas

The same `SplitPanel` widget (or `Panel` or `Widget`) can be moved to the left area:

In [29]:
split_panel.add_to_shell(area=ipylab.Area.left, rank=1000)
await split_panel.connections[0].activate()

Or to the right area:

In [30]:
split_panel.add_to_shell(area=ipylab.Area.right, rank=1000)
await split_panel.connections[0].activate()

In [31]:
await app.shell.collapse_right()

Notice how it moved the widget instead of adding a second one?

This is the default behaviour.

To have multiple widgets, provide it with a new `cid` when 'adding' it to the shell.

In [32]:
await split_panel.add_to_shell(cid=ipylab.ShellConnection.to_cid(), mode=ipylab.InsertMode.split_right)

< Not ready: ShellConnection(cid='ipylab-ShellConnection|ba333e5f-5d65-4bdb-8d47-4ab6df532677') >

In [33]:
split_panel.connections[0].activate()
split_panel.connections

(ShellConnection(cid='ipylab-ShellConnection|d3aa35a1-f0a0-4617-9fb8-0e77cc48f996'),
 < Not ready: ShellConnection(cid='ipylab-ShellConnection|ba333e5f-5d65-4bdb-8d47-4ab6df532677') >)

In [34]:
split_panel.close()
fail
await asyncio.sleep(1)

NameError: name 'fail' is not defined

In [ ]:
split_panel.connections